In [ ]:
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    load_person_assign_day,
    load_tnc_trips,
    out_dir,
    prep_data_1d,
    tour_extract_wkday_dir,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell


In [ ]:
out_filepath = out_dir / "07_Persons_AvgTNCTrips.xlsx"

In [ ]:
person = load_person_assign_day(
    tour_extract_wkday_dir / "person-assign_day.csv", demographics=False
).to_pandas()
trip = load_tnc_trips(tour_extract_wkday_dir)

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})
format2 = workbook.add_format({"num_format": "0.000"})

In [ ]:
row = 0
sname = "Weekday"
d1_dict = df_to_excel_col_dict["tnc_type"]
PREF = "Daily Avg TNC Trips"

tab = prep_data_1d(
    trip,
    d1_dict["desc"],
    d1_dict["col"],
    "trexpfac",
    d1_dict["vals"],
    d1_dict["labels"],
)


def prep_avg(suff, num_per):
    df = tab[[d1_dict["desc"] + "_" + suff]]
    df["persons_" + suff] = num_per
    df["AvgDaily_TNCTrips"] = df[d1_dict["desc"] + "_" + suff] / df["persons_" + suff]
    return df


title = PREF + " (Unweighted)"
out_tab = prep_avg("samp", len(person))
row = write_to_excel(writer, out_tab, sname, title, row)

title = PREF + " (Weighted)"
out_tab = prep_avg("wtd", sum(person["psexpfac"]))
row = write_to_excel(writer, out_tab, sname, title, row)

tab_range = xl_rowcol_to_cell(0, 1) + ":" + xl_rowcol_to_cell(0, 2)
_ = writer.sheets[sname].set_column(tab_range, 11, format1)
tab_range = xl_rowcol_to_cell(0, 3) + ":" + xl_rowcol_to_cell(0, 3)
_ = writer.sheets[sname].set_column(tab_range, 11, format2)

In [ ]:
writer.close()